In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# IWB temp data from online source
weatherbuoy_df = pd.read_csv('https://erddap.marine.ie/erddap/tabledap/IWBNetwork.csvp?station_id%2Ctime%2CAtmosphericPressure%2CWindDirection%2CWindSpeed%2CWaveHeight%2CWavePeriod%2CSeaTemperature%2Csalinity')

# from local source
# weatherbuoy_df = pd.read_csv('data\IWBNetwork_archive.csv')
weatherbuoy_df['time (UTC)'] = pd.to_datetime(weatherbuoy_df['time (UTC)'])
weatherbuoy_df['year'] = weatherbuoy_df['time (UTC)'].dt.year
weatherbuoy_df['month'] = weatherbuoy_df['time (UTC)'].dt.month
weatherbuoy_df['day'] = weatherbuoy_df['time (UTC)'].dt.day
weatherbuoy_df['mean_yr_station'] = weatherbuoy_df.groupby(['year', 'station_id'])[
    "SeaTemperature (degrees_C)"].transform('mean')
weatherbuoy_df['min_yr_station'] = weatherbuoy_df.groupby(['year', 'station_id'])[
    "SeaTemperature (degrees_C)"].transform('min')
weatherbuoy_df['max_yr_station'] = weatherbuoy_df.groupby(['year', 'station_id'])[
    "SeaTemperature (degrees_C)"].transform('max')
# convert number months to the text months for easier display
weatherbuoy_df['months'] = weatherbuoy_df['time (UTC)'].dt.strftime('%b')

In [ ]:
filtered_weatherbuoy = weatherbuoy_df[weatherbuoy_df['station_id'].isin(['M1', 'M2', 'M3', 'M4', 'M5', 'M6'])]
# filter for anything greater than 0 as it looks like there is an outlier with a value of 0
filtered_weatherbuoy = filtered_weatherbuoy[filtered_weatherbuoy['SeaTemperature (degrees_C)'] > 0]

In [ ]:
plt.figure(figsize=(16,10))
# Theme
sns.set_theme(style="dark", rc={"axes.facecolor": (0, 0, 0, 0), 'axes.linewidth':0})
# palette = sns.color_palette("Plasma", 12) 
# create a grid with a row for each 'month'
g = sns.FacetGrid(filtered_weatherbuoy, 
                  palette='inferno',
                  row="months", 
                  hue="months", 
                  row_order=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                  aspect=9, 
                  height=0.8) 
g.map_dataframe(sns.kdeplot, 
                x="SeaTemperature (degrees_C)", 
                cmap='plasma',
                fill=True, alpha=1)

def label(x, color, label):
    ax = plt.gca() #get current axis
    ax.text(0, .2, label, color='black', fontsize=13,
    ha="left", va="center", transform=ax.transAxes)
    
# iterate grid to plot labels
g.map(label, "months")
# adjust subplots to create overlap
g.fig.subplots_adjust(hspace=-.3) 
# remove subplot titles
g.set_titles("") 
# remove yticks and labels and set xlabel
g.set(yticks=[], ylabel="", xlabel="Sea Temperature (degrees Celcius)")
# remove left spine
g.despine(left=True)
# set title
plt.suptitle('Distribution of Sea Temperatures around the Irish Coast', y=0.98)
plt.savefig('../content/sea_temp_ridgeplot.png')